In [194]:
from math import exp
import pandas as pd
import numpy as np
from scipy.special import gamma as GAMMA

In [195]:
#zeta, tau, gamma = 0.35563196, -0.53661156,  1.53815421
zeta, tau, gamma =0.69538854, -0.62408302, -0.08214939

In [196]:
def get_fractional_series(X0):
    """
    :param X0: 待计算分数阶的序列
    :return: 分数阶序列
    """
    len_x0 = len(X0)

    # 初始化数组
    xishu = np.zeros((len_x0, len_x0))
    Xr = np.zeros(len_x0)

    for k in range(len_x0):
        tmp = 0
        for i in range(k + 1):
            xishu[k, i] = GAMMA(zeta + k - i) / (GAMMA(k - i + 1) * GAMMA(zeta))
            tmp += xishu[k, i] * X0[i]
        Xr[k] = tmp

    return Xr

In [197]:
data = pd.read_excel("使用版2005-2023-2个相关因素_2位小数.xlsx")
data = data.iloc[11:, :] # i=3
n = data.shape[0]
train_n = n-2

In [198]:
# 第一列是年份，故先舍弃
X0 = data.iloc[:train_n, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T
y1 = X1[:, 0] ** (1 - gamma)

Zy = (y1[1:] + y1[:-1]) / 2
M2 = (X1[1:, 1] + X1[:-1, 1]) / 2
M3 = (X1[1:, 2] + X1[:-1, 2]) / 2

In [199]:
# 构建 B
TAU = (np.exp(tau) - 1) / tau * np.exp(tau * (np.arange(1, train_n)))
ONE = np.ones(train_n - 1)
B = (1 - gamma) * np.array([-Zy, M2, M3, TAU, ONE]).T

# 构建 Y
Y = np.diff(y1)

# 最小二乘法
a, b2, b3, h1, h2 = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Y)
a, b2, b3, h1, h2

(2.335350771783407,
 -0.8119895152819936,
 420.69153137870575,
 -428108.2651795752,
 506637.39270007284)

In [200]:
# 求 mu
mu1 = (1-gamma)/(1+0.5*a*(1-gamma))
mu2 = (1-0.5*a*(1-gamma))/(1+0.5*a*(1-gamma))
mu3 = (h1*(1-gamma)*(exp(tau)-1))/(1+0.5*a*(1-gamma))/tau
mu4 = (h2*(1-gamma))/(1+0.5*a*(1-gamma))
mu1,mu2,mu3,mu4

(0.4780658107952783,
 -0.11645136020401342,
 -152246.92389441698,
 242206.0159203662)

In [201]:

# 这里重新获取下原数据，以实现所有数据的预测
# 第一列是年份，故先舍弃
X0 = data.iloc[:, 1:].to_numpy()

X1 = np.array([get_fractional_series(X0[:,0]), get_fractional_series(X0[:,1]), get_fractional_series(X0[:,2])]).T

# 求 hat y1(1为上标的那个haty^1)
hat_y1 = [y1[0]]
for m in range(2, n + 1):
    hat_ym = 0
    hat_ym += mu2 ** (m - 1) * y1[0]
    for v in range(1, m):
        hat_ym += mu2 ** (v - 1) * mu1 * (b2 * X1[m - v, 1] + b3 * X1[m - v, 2])
    for w in range(0, m - 1):
        hat_ym += mu2 ** w * (mu4 + mu3 * exp(tau * (m - w - 2)))
    hat_y1.append(hat_ym)
print(hat_y1)

[222300.6555879836, 420816.52735969034, 630885.9155247337, 812298.8795289613, 1026329.2632339494, 1303430.2552394054, 1607683.9882711377, 1898096.352780945]


In [202]:


hat_x1 = np.array(hat_y1) ** (1 / (1 - gamma))
print(hat_x1)
hat_x0 = []
for k in range(2, n + 1):
    hat_x0_m = 0
    for j in range(0, k):
        hat_x0_m += (-1) ** j * GAMMA(zeta + 1) / (GAMMA(j + 1) * GAMMA(zeta - j + 1)) * hat_x1[k - j - 1]
    hat_x0.append(hat_x0_m)
print(hat_x0)

[ 87304.68       157452.68807087 228906.52986996 289128.29369623
 358881.3461068  447581.47330229 543335.80104089 633448.36165294]
[96742.01411050715, 110169.15147694996, 109252.17437777638, 124012.29179912913, 151145.08401259582, 170832.06834842323, 177184.7341926856]


In [203]:
original_x1 = data.iloc[1:, 1]
APE = np.abs(hat_x0 - original_x1) / original_x1
MAPE=np.mean(APE)
print(APE)
print(MAPE)

12    0.040577
13    0.116944
14    0.091452
15    0.126504
16    0.136530
17    0.145849
18    0.157715
Name: 主营业务收入（亿元）, dtype: float64
0.11651006945980294


In [204]:
pd.DataFrame(hat_x0).to_excel('FNGBM_hatx0_case1-11.xlsx')
pd.DataFrame(APE).to_excel('FNGBM_APE-case1-11.xlsx')